In [1]:
import pandas as pd
import numpy as np

all_feature = pd.read_csv('/kaggle/input/abc-feature-labels/all_features(with jerk).csv')
all_label = pd.read_csv('/kaggle/input/abc-feature-labels/all_labels(with jerk).csv')

In [2]:
all_feature = all_feature.to_numpy()
print(all_feature.shape)
print(all_label.shape)
all_label = all_label.to_numpy()
all_label = all_label.T.flatten()
print(all_label.shape)

from sklearn.ensemble import RandomForestClassifier

(9716, 679)
(9716, 1)
(9716,)


In [3]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [4]:
# Define RFC
model = RandomForestClassifier(random_state=42)

# Create Pipeline
steps = list()
steps.append(('scaler', MinMaxScaler()))
steps.append(('classifier', model))
pipeline = Pipeline(steps=steps)

# Define Parameters

params= {
    'classifier__n_estimators' : [10, 25, 50, 75, 100, 150, 200, 250],
    'classifier__max_depth':[5, 10, 15, 20, 25],
    'classifier__class_weight':[None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]    
}

In [5]:
#Random search for ideal n_estimators
print(f'Searching for the best value of parameter for {model}...')
grid = RandomizedSearchCV(pipeline, params, cv=5, n_jobs=-1, scoring='f1_macro').fit(all_feature, all_label)

Searching for the best value of parameter for RandomForestClassifier(random_state=42)...


In [6]:
classifier=grid
print(classifier.cv_results_)
print(classifier.best_params_)
print(classifier.best_score_)

{'mean_fit_time': array([40.74233079, 49.88139591,  5.05363913, 40.03794713,  3.92005234,
       21.19904299,  2.14692869, 50.54824934, 29.4718709 , 14.02124538]), 'std_fit_time': array([1.05089808, 0.95311378, 0.09580747, 0.9712272 , 0.17493475,
       0.23568876, 0.02655894, 1.13566141, 0.57985547, 1.02313142]), 'mean_score_time': array([0.17955923, 0.22119946, 0.03514996, 0.16962533, 0.03243303,
       0.15020971, 0.02756982, 0.2023448 , 0.14620948, 0.05947042]), 'std_score_time': array([0.00350147, 0.00920314, 0.00125532, 0.0069506 , 0.00135146,
       0.00601012, 0.00070589, 0.00818563, 0.00554526, 0.00947863]), 'param_classifier__n_estimators': masked_array(data=[200, 250, 25, 200, 25, 250, 25, 250, 200, 75],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_classifier__max_depth': masked_array(data=[20, 20, 20, 20, 10, 5, 5, 25, 10, 20],
             mask=[False, F

In [9]:
#{'classifier__n_estimators': 150, 'classifier__max_depth': 25, 'classifier__class_weight': {0: 1, 1: 5}}
#0.49188742341750713
model_final = RandomForestClassifier(n_estimators=150, max_depth=20,class_weight={0:1,1:5},random_state=42)
#model_final.set_params(**classifier.best_params_)


# Split train-test data
f_train,f_test,l_train,l_test = train_test_split(all_feature, all_label, test_size = 0.2, random_state=11, shuffle = True)

# Scale data
scaler = MinMaxScaler()
scaler.fit(f_train)
f_train = scaler.transform(f_train)
f_test = scaler.transform(f_test)


# Fit + Train data
model_final.fit(f_train,l_train)
prediction = model_final.predict(f_test)

# Evaluate
print(classification_report(l_test,prediction))


              precision    recall  f1-score   support

           0       0.79      0.93      0.86       542
           1       0.71      0.38      0.49        32
           2       0.93      0.90      0.92       185
           3       0.73      0.49      0.59        55
           4       0.89      0.71      0.79       352
           5       0.95      0.36      0.52        53
           6       0.83      0.60      0.70        50
           7       0.84      0.97      0.90       536
           8       0.97      0.82      0.89       139

    accuracy                           0.85      1944
   macro avg       0.85      0.68      0.74      1944
weighted avg       0.85      0.85      0.84      1944

